In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,GRU,Dense,AdditiveAttention,StringLookup,TextVectorization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import csv

In [5]:
from google.colab import drive
drive.mount('/content/drive')

with open("/content/drive/MyDrive/data/eng-ind.csv") as f:
  csv_reader = csv.reader(f)
  next(csv_reader)

  input_data = list()
  target_data = list()

  for row in csv_reader:
    input_data.append(row[0])
    target_data.append(row[1])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
BUFFER_SIZE = len(input_data)
BATCH_SIZE = 64

dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [ ]:
for input_batch, target_batch in dataset.take(1):
  print(input_batch,"\n")
  print(target_batch)
  break

In [8]:
def tf_lower_and_split_punct(text):
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [10]:
max_vocab_size = 5000

input_text_processor = TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

output_text_processor = TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=max_vocab_size)

In [11]:
input_text_processor.adapt(input_data)
output_text_processor.adapt(target_data)